<h2>Importando a biblioteca pandas</h2> 

In [ ]:
import pandas as pd

<h3> buscando os dados e alocando da variável dados</h3>


In [ ]:
dados  = pd.read_csv(r"ArvoresDecisao\dados\creditcard.csv")

In [4]:
n_transacoes = dados['Class'].count()
n_fraudes    = dados['Class'].sum()
n_normais    = n_transacoes - n_fraudes
fraudes_porc = n_fraudes / n_transacoes
nromais_porc = n_normais / n_transacoes

print('numero de transacoes ',n_transacoes )
print('numero de fraudes ',n_fraudes, "%.2f"% (fraudes_porc *100))
print('numero de transacoes normais ',n_normais ,"%.2f"% (nromais_porc *100))

numero de transacoes  284807
numero de fraudes  492 0.17
numero de transacoes normais  284315 99.83


In [5]:
# Por conta dos dados serem desbalanceados usaremos o StratifiedShuffleSplit para garantir 
# Que usaremos a porcentagem de amostra equivalentes ao treino e teste

from sklearn.model_selection import StratifiedShuffleSplit

def executar_validador(X,y):
    validador = StratifiedShuffleSplit( n_splits= 1,test_size=0.1,random_state=0 )
    
    for treino_id,teste_id in validador.split(X,y):
        X_train, X_test  = X[treino_id],X[teste_id]
        y_train, y_test  = y[treino_id],y[teste_id]
    
    return X_train,X_test,y_train,y_test

In [6]:
%%time
from sklearn import tree
def executar_classificador(classificador,X_train, X_test, y_train):
    arvore = classificador.fit(X_train, y_train)
    y_pred = arvore.predict(X_test)
    return y_pred

Wall time: 0 ns


In [7]:
import matplotlib.pyplot as plt

def salvar_arvore(classificador, nome):
    plt.figure(figsize = (200,100))
    tree.plot_tree(classificador,filled=True, fontsize =14)
    plt.savefig(nome)
    plt.close()

In [8]:
from sklearn.metrics import accuracy_score   # responsavel por fazer a acuracia da arvore
from sklearn.metrics import confusion_matrix # responsavel por mostrar quantos acertos temos dentro da acuracia
from sklearn.metrics import precision_score  #
from sklearn.metrics import recall_score  #

def validar_arvore(y_test,y_pred):
    print(accuracy_score(y_test,y_pred))
    print(precision_score(y_test,y_pred))
    print(recall_score(y_test,y_pred))
    print(confusion_matrix (y_test,y_pred))

In [9]:
#execucao do validador
X = dados.drop('Class', axis =1).values #representa todos os atributos
y = dados['Class'].values # rotuilos valores da classe
X_train,X_test,y_train,y_test = executar_validador(X,y)

In [10]:
#execucao do classificador DecisionTreeClassifier
classificador_arvore_decisao = tree.DecisionTreeClassifier()
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao,X_train,X_test,y_train)

In [11]:
#criacao da figura da arvore de decisao
salvar_arvore(classificador_arvore_decisao,'arvore_decisao1.png')

In [12]:
#validacao a arvore de decisao
validar_arvore(y_test,y_pred_arvore_decisao)

#0.9990871107053826
#0.7551020408163265 com fraude
#[
#    [28418    12] - numero acerto ( sem fraude )  || numero erros ( sem fraude ) -- y_pred
#    [   14    37] -  numero erros (fraude )      ||  numero  acerto  ( fraude ) -- y_test
#]

0.9991222218320986
0.7608695652173914
0.7142857142857143
[[28421    11]
 [   14    35]]


In [13]:
#execucao do classificador DecisionTreeClassifier
classificador_arvore_decisao = tree.DecisionTreeClassifier(max_depth = 10, random_state = 0)
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao,X_train,X_test,y_train)

In [14]:
validar_arvore(y_test,y_pred_arvore_decisao)

0.9994733330992591
0.9473684210526315
0.7346938775510204
[[28430     2]
 [   13    36]]


In [15]:
#execucao do classificador DecisionTreeClassifier
classificador_arvore_decisao = tree.DecisionTreeClassifier(max_depth = 10, random_state = 0, min_samples_leaf = 10)
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao,X_train,X_test,y_train)

In [16]:
validar_arvore(y_test,y_pred_arvore_decisao)

0.9993679997191109
0.8604651162790697
0.7551020408163265
[[28426     6]
 [   12    37]]


In [18]:
%%time
from sklearn.ensemble import RandomForestClassifier

classificador_random_forest = RandomForestClassifier (n_estimators = 100, random_state =0)
y_pred_random_forest =  executar_classificador(classificador_random_forest,X_train,X_test,y_train)

Wall time: 3min 19s


In [19]:
salvar_arvore(classificador_random_forest.estimators_[0],"random_florest1")
salvar_arvore(classificador_random_forest.estimators_[1],"random_florest2")

In [23]:
validar_arvore(y_test,y_pred_random_forest)

0.9995435553526912
0.9736842105263158
0.7551020408163265
[[28431     1]
 [   12    37]]


In [22]:
%%time
from sklearn.ensemble import RandomForestClassifier

classificador_random_forest = RandomForestClassifier (n_estimators = 50, random_state =0, max_depth =10)
y_pred_random_forest =  executar_classificador(classificador_random_forest,X_train,X_test,y_train)

Wall time: 59.9 s


In [27]:
%%time
from sklearn.ensemble import AdaBoostClassifier
classificador_adaboost = AdaBoostClassifier(random_state = 0)
y_pred_adaboost = executar_classificador(classificador_adaboost, X_train, X_test, y_train)


Wall time: 58.2 s


In [28]:
salvar_arvore(classificador_adaboost.estimators_[0], 'adaboost1')
salvar_arvore(classificador_adaboost.estimators_[1], 'adaboost2')
validar_arvore(y_test,y_pred_adaboost)

0.9992626663389628
0.8888888888888888
0.6530612244897959
[[28428     4]
 [   17    32]]


In [29]:
%%time
from sklearn.ensemble import AdaBoostClassifier
classificador_adaboost = AdaBoostClassifier(random_state = 0, n_estimators = 100)
y_pred_adaboost = executar_classificador(classificador_adaboost, X_train, X_test, y_train)

Wall time: 1min 58s


In [30]:
validar_arvore(y_test,y_pred_adaboost)

0.999403110845827
0.8636363636363636
0.7755102040816326
[[28426     6]
 [   11    38]]


In [31]:
%%time
from sklearn.ensemble import AdaBoostClassifier
classificador_adaboost = AdaBoostClassifier(random_state = 0, n_estimators = 200)
y_pred_adaboost = executar_classificador(classificador_adaboost, X_train, X_test, y_train)

Wall time: 4min 3s


In [32]:
validar_arvore(y_test,y_pred_adaboost)

0.9995435553526912
0.9285714285714286
0.7959183673469388
[[28429     3]
 [   10    39]]
